In [ ]:
# install tensorflow
!pip install tensorflow==2.15.0

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:

train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

# Convert labels to binary (0 for ham, 1 for spam)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Split into features and labels
train_messages = train_df['message'].values
train_labels = train_df['label'].values
test_messages = test_df['message'].values
test_labels = test_df['label'].values

# Define the TextVectorization layer
max_features = 20000  # Increased from 10000 to capture more vocabulary
sequence_length = 50   # Reduced from 100 as SMS messages are typically short

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

# Adapt the vectorization layer to the training text
vectorize_layer.adapt(train_messages)

# Build the model with increased complexity
model = keras.Sequential([
    vectorize_layer,
    keras.layers.Embedding(max_features + 1, 64),  # Increased embedding dimension
    keras.layers.Dropout(0.2),  # Added dropout for regularization
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation='relu'),  # Increased units in dense layer
    keras.layers.Dropout(0.2),  # Added dropout
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# Train the model with more epochs
history = model.fit(
    train_messages,
    train_labels,
    epochs=20,  # Increased from 10 to 20
    validation_data=(test_messages, test_labels),
    verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(test_messages, test_labels)
print(f"Test Accuracy: {accuracy}")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prediction = model.predict([pred_text])
    pred_prob = prediction[0][0]
    pred_label = 'spam' if pred_prob > 0.5 else 'ham'
    return [float(pred_prob), pred_label]
    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
